This file export the train and test set prediction vs. actual for a single protein for a model


In [1]:

import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from predict_protein import download_cptac_original_order, select_features, train_model, utils

model_loc = '/Volumes/Ginkgo_Data/Data/2021-11_CPTACTrainedModels/out3'


In [2]:
cptac_list = download_cptac_original_order.download_cptac(n_tumors=8)
tumor_df = pd.concat(cptac_list)
tm = train_model.LearnCPTAC(tumor_df)


cptac warning: Your version of cptac (1.1.0) is out-of-date. Latest is 1.1.2. Please run 'pip install --upgrade cptac' to update it. (/usr/local/Cellar/python@3.9/3.9.7_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py, line 910)


cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac_original_order.py, line 38)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac_ori

In [3]:
def get_predictions(tm_obj, model_loc, model, protein_todo):

    with open(model_loc, 'rb') as f:
        model_results = pickle.load(f)

    protein_model  = [result['model'] for result in model_results if result['metrics'].index[0] == protein_todo][0]

    # Get features and then do train test split again
    tm_obj.included_features = model
    print(tm_obj.included_features)
    x_train, x_test, y_train, y_test = tm_obj.get_train_test(protein_to_do=protein_todo)

    # Output a pandas data frame with y (actual mass spec protein level) and y_hat (predicted from transcript(s))
    train_df = pd.DataFrame({'y': y_train, 'yhat': protein_model.predict(x_train[protein_model.feature_names_in_]), 'set': 'train'})
    test_df = pd.DataFrame({'y': y_test, 'yhat': protein_model.predict(x_test[protein_model.feature_names_in_]), 'set': 'test'})

    return pd.concat([train_df, test_df])


In [4]:
for model in ['single', 'corum', 'stringhi', 'string', 'all']: # 'single', 'corum', 'stringhi', 'string'
    for protein in ['CMC1', 'PCCB', 'PSMG2', 'SMCR8', 'MICU2', 'PPP3R1']:
        df = get_predictions(tm_obj=tm,
                             model_loc=f'{model_loc}/tumor8_elastic_{model}.p',
                             model=model,
                             protein_todo=protein,
                        )
        df.to_csv(f'07_ExportSingleProtein_Output/{protein}_{model}_out3.csv', index=False)

single
single
single
single
single
single
corum
corum
corum
corum
corum
corum
stringhi
stringhi
stringhi
stringhi
stringhi
stringhi
string
string
string
string
string
string
all
all
all
all
all
all


In [5]:
tm.df

Name,A1BG-AS1_transcriptomics,A1CF_transcriptomics,A2M_transcriptomics,A2M-AS1_transcriptomics,A2ML1_transcriptomics,A2MP1_transcriptomics,A3GALT2_transcriptomics,A4GALT_transcriptomics,A4GNT_transcriptomics,AA06_transcriptomics,...,ZNF721_proteomics,ZNF723_proteomics,ZNF724_proteomics,ZNF75D_proteomics,ZNF774_proteomics,ZNF836_proteomics,ZNF85_proteomics,ZNF880_proteomics,ZNF91_proteomics,ZSCAN5A_proteomics
Patient_ID,,,,,,,,,,,,,,,,,,,,,
OV01OV002,-1.328216,0.349336,0.082808,NaN,0.420501,-0.809248,-0.138877,0.388107,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OV01OV007,-0.676969,NaN,0.614237,NaN,-0.603361,NaN,NaN,0.367057,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OV01OV008,-2.093132,NaN,-0.096348,NaN,-0.494284,NaN,NaN,0.197533,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OV01OV010,0.428657,NaN,0.671921,NaN,0.993398,NaN,NaN,-0.224448,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OV01OV013,-0.975500,NaN,0.375882,NaN,-1.411217,0.308969,NaN,-0.147564,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LSC3N-03886.N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.7562,NaN,NaN,0.4721,2.1518,NaN,0.0548,NaN,NaN
LSC3N-04124.N,-0.274753,NaN,0.267354,1.120134,-0.281525,1.008742,-0.127427,-0.547766,0.283572,NaN,...,-0.4921,0.4016,0.3106,NaN,-0.8536,1.9444,-0.5946,-1.1269,0.3020,NaN
LSC3N-04127.N,-0.035043,1.065609,0.401612,0.625109,-0.357490,0.830724,0.194589,-0.351362,0.575559,NaN,...,NaN,-1.5687,NaN,NaN,-0.4416,NaN,NaN,-0.4127,-1.6368,-1.3511
